In [204]:
import pandas as pd
import numpy as np
#importing necessary libraries

In [205]:
Badges = pd.read_csv("Badges.csv")
#saving csv files to variable

In [206]:
import os, os.path
import sqlite3
import tempfile
# path to database file, I am just creating not existing one
baza = os.path.join(tempfile.mkdtemp(), 'example.db')
if os.path.isfile(baza): # if this file already exists...
    os.remove(baza) # ...we will remove it
conn = sqlite3.connect(baza) # create the connection
Badges.to_sql("Badges", conn) # import the data frame into the database



In [207]:
referenceSolution1=pd.read_sql_query("""
SELECT Name,
COUNT(*) AS Number,
MIN(Class) AS BestClass
FROM Badges
GROUP BY Name
ORDER BY Number DESC
LIMIT 10
""", conn)
# reference solution 1

In [208]:
Badges
# Let's see how badges looks like!

,Id,UserId,Name,Date,Class,TagBased
0,1,2,Autobiographer,2011-06-21T20:16:48.910,3,False
1,2,3,Autobiographer,2011-06-21T20:16:48.910,3,False
2,3,4,Autobiographer,2011-06-21T20:16:48.910,3,False
3,4,6,Autobiographer,2011-06-21T20:21:49.157,3,False
4,5,8,Autobiographer,2011-06-21T20:21:49.157,3,False
...,...,...,...,...,...,...
193587,223612,45793,Citizen Patrol,2021-09-05T02:21:49.590,3,False
193588,223613,122822,Autobiographer,2021-09-05T02:56:55.223,3,False
193589,223614,1375,Nice Answer,2021-09-05T02:56:55.223,3,False
193590,223615,122803,Popular Question,2021-09-05T04:27:00.960,3,False


In [209]:
res = Badges.groupby('Name').agg(Number=('Class','size'),BestClass=('Class','min'))
# We want to group by a Name variable, and then see Count of rows and minimal Class
# I use size for class, because there are equally many rows with Class variable as number of rows for every Name

In [210]:
res = res.sort_values(by='Number', ascending=False)
# sorting by Number

In [211]:
res=res.head(10)
# Gettnig first 10 results

In [212]:
res=res.reset_index(level=None,drop=False)
# resetting an index

In [213]:
res

,Name,Number,BestClass
0,Autobiographer,24564,3
1,Student,23930,3
2,Supporter,17826,3
3,Popular Question,14840,3
4,Nice Answer,12406,3
5,Yearling,11281,2
6,Teacher,9746,3
7,Editor,9115,3
8,Notable Question,8613,2
9,Scholar,7224,3


In [214]:
res.equals(referenceSolution1)
# Let's check if the output matches reference solution

True

In [215]:
Users = pd.read_csv("Users.csv")
# reading Users csv

In [216]:
Users.to_sql("Users", conn)
# loading table to database

In [217]:
Posts = pd.read_csv("Posts.csv")
# reading Posts csv

In [218]:
Posts.to_sql("Posts", conn)
# loading table to database

In [219]:
referenceSolution2=pd.read_sql_query("""
SELECT Location, COUNT(*) AS Count
FROM (
SELECT Posts.OwnerUserId, Users.Id, Users.Location
FROM Users
JOIN Posts ON Users.Id = Posts.OwnerUserId
)
WHERE Location NOT IN ('')
GROUP BY Location
ORDER BY Count DESC
LIMIT 10
""", conn)
# reference solution for task 2

In [220]:
referenceSolution2

,Location,Count
0,"Christchurch, New Zealand",2765
1,"New York, NY",1788
2,"London, United Kingdom",1708
3,UK,1590
4,"Sunshine Coast QLD, Australia",1550
5,Australia,1183
6,"Vancouver, Canada",967
7,Netherlands,935
8,on the server farm,924
9,Pennsylvania,921


In [221]:
UsersT2= Users[['Id','Location']]
# getting useful fields

In [222]:
InnerT2= UsersT2.merge(Posts,how='inner',left_on='Id',right_on='OwnerUserId')
# merging it with Posts table by Id=OwnerUserId

In [223]:
InnerT2=InnerT2[['OwnerUserId','Location']]
# Getting useful fields

In [224]:
InnerT2

,OwnerUserId,Location
0,-1.0,on the server farm
1,-1.0,on the server farm
2,-1.0,on the server farm
3,-1.0,on the server farm
4,-1.0,on the server farm
...,...,...
112914,122803.0,NaN
112915,122805.0,NaN
112916,122815.0,NaN
112917,122819.0,NaN


In [225]:
InnerT2=InnerT2.dropna()
# equal to Location!=''

In [226]:
InnerT2

,OwnerUserId,Location
0,-1.0,on the server farm
1,-1.0,on the server farm
2,-1.0,on the server farm
3,-1.0,on the server farm
4,-1.0,on the server farm
...,...,...
112903,122742.0,"Toronto, ON, Canada"
112906,122766.0,"Moscow, Russia"
112910,122774.0,"Chennai, Tamil Nadu, India"
112912,122780.0,Hong Kong


In [227]:
res2 = InnerT2.groupby('Location').agg(Count=('OwnerUserId','size'))
# grouping by Location and counting rows

In [228]:
res2 = res2.sort_values(by='Count', ascending=False)
# sorting

In [229]:
res2

,Count
Location,
"Christchurch, New Zealand",2765
"New York, NY",1788
"London, United Kingdom",1708
UK,1590
"Sunshine Coast QLD, Australia",1550
...,...
Luxembourg & Heidelberg,1
Luton,1
"Ludwigshafen am Rhein, Germany",1


In [230]:
res2=res2.reset_index(level=None,drop=False)
# I set an index to be a column

In [231]:
res2=res2.head(10)
# getting first 10 rows

In [232]:
res2.equals(referenceSolution2)
# checking if it is the same as referential solution

True

In [233]:
referenceSolution3 = pd.read_sql_query("""
SELECT
Users.AccountId,
Users.DisplayName,
Users.Location,
AVG(PostAuth.AnswersCount) as AverageAnswersCount
FROM
(
SELECT
AnsCount.AnswersCount,
Posts.Id,
Posts.OwnerUserId
FROM (
SELECT Posts.ParentId, COUNT(*) AS AnswersCount
FROM Posts
WHERE Posts.PostTypeId = 2
GROUP BY Posts.ParentId
) AS AnsCount
JOIN Posts ON Posts.Id = AnsCount.ParentId
) AS PostAuth
JOIN Users ON Users.AccountId=PostAuth.OwnerUserId
GROUP BY OwnerUserId
ORDER BY AverageAnswersCount DESC
LIMIT 10
""", conn)
# setting up referential solution

In [234]:
referenceSolution3

,AccountId,DisplayName,Location,AverageAnswersCount
0,40811.0,vocaro,"San Jose, CA",11.0
1,280.0,csmba,"San Francisco, CA",11.0
2,44093.0,Emma Arbogast,"Salem, OR",10.0
3,204.0,Josh,Australia,10.0
4,11758.0,rvarcher,"Oklahoma City, OK",9.0
5,79346.0,Thomas Matthews,California,8.0
6,54571.0,Christian,None,8.0
7,42364.0,Petrogad,None,8.0
8,20473.0,Jeremy Boyd,"Houston, TX",8.0
9,19588.0,JD Isaacks,"Atlanta, GA",8.0


In [235]:
InnerInnerT3=Posts[Posts['PostTypeId']==2]
# setting filter of PostTypeId==2

In [236]:
AnsCount = InnerInnerT3.groupby('ParentId').agg(AnswersCount=('Id','size'))
# grouping by and counting rows

In [237]:
AnsCount

,AnswersCount
ParentId,
1.0,4
2.0,8
4.0,1
5.0,5
6.0,6
...,...
168517.0,2
168518.0,2
168534.0,4


In [238]:
AnsCount=AnsCount.reset_index(level=None,drop=False)
# index to column one more time

In [239]:
PostAuth = AnsCount.merge(Posts,how='inner',left_on='ParentId',right_on='Id')
# merging tables

In [240]:
PostAuth = PostAuth[['AnswersCount','Id','OwnerUserId']]
# getting desirable fields

In [241]:
PostAuth

,AnswersCount,Id,OwnerUserId
0,4,1,9.0
1,8,2,13.0
2,1,4,24.0
3,5,5,13.0
4,6,6,19.0
...,...,...,...
38158,2,168517,107806.0
38159,2,168518,1810.0
38160,4,168534,122803.0
38161,1,168537,122805.0


In [242]:
res3 = PostAuth.merge(Users,how='inner',left_on='OwnerUserId',right_on='AccountId')
# merging Users with PostAuth

In [243]:
res3 = res3.groupby('AccountId').agg(AverageAnswersCount=('AnswersCount','mean'))
# groupnig by Account Id to get the average

In [244]:
res3 = res3.reset_index(level=None,drop=False)
# index to column one more time

In [245]:
res3 = res3.sort_values(by=['AverageAnswersCount','AccountId'], ascending=False)
# sorting by average and accountId number, it is not specified but I have to do it

In [246]:
res3 = res3.head(10)
# getting first 10 rows

In [247]:
res3 = res3.merge(Users,how='inner',on='AccountId')
# merging with Users to get DisplayName, Location fields

In [248]:
res3 = res3[['AccountId','DisplayName','Location','AverageAnswersCount']]
# getting fields

In [249]:
res3

,AccountId,DisplayName,Location,AverageAnswersCount
0,40811.0,vocaro,"San Jose, CA",11.0
1,280.0,csmba,"San Francisco, CA",11.0
2,44093.0,Emma Arbogast,"Salem, OR",10.0
3,204.0,Josh,Australia,10.0
4,11758.0,rvarcher,"Oklahoma City, OK",9.0
5,79346.0,Thomas Matthews,California,8.0
6,54571.0,Christian,NaN,8.0
7,42364.0,Petrogad,NaN,8.0
8,20473.0,Jeremy Boyd,"Houston, TX",8.0
9,19588.0,JD Isaacks,"Atlanta, GA",8.0


In [250]:
res3.equals(referenceSolution3)
# checking if its good

True

In [251]:
conn.close()
# closing database connection